# Execute this block if your running this notebook in Colab

In [ ]:
!gdown 1iOQziZqU0v7eGaHnCZmdKEOHJP5idc_y
!mkdir /root/.kaggle/
!mv ./kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!pip install -q kaggle
!kaggle competitions download -c tabular-playground-series-aug-2022
!unzip tabular-playground-series-aug-2022.zip

Downloading...
From: https://drive.google.com/uc?id=1iOQziZqU0v7eGaHnCZmdKEOHJP5idc_y
To: /content/kaggle.json
100% 64.0/64.0 [00:00<00:00, 81.9kB/s]
  0% 0.00/2.27M [00:00<?, ?B/s]
100% 2.27M/2.27M [00:00<00:00, 36.2MB/s]
Archive:  tabular-playground-series-aug-2022.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Start from here if your running this notebook in other environments

In [ ]:
!pip install feature_engine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 KB 9.8 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.layers import Input, BatchNormalization, Dense, Dropout, concatenate
from tensorflow.nn import sigmoid
from tensorflow.keras.activations import swish
from tensorflow.keras.models import Model
import tensorflow as tf
from sklearn.linear_model import HuberRegressor
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from feature_engine.encoding import WoEEncoder
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [ ]:
# [ref] https://www.kaggle.com/code/purist1024/principled-3-vs-2-cv-splitting-on-product-code
folds_dict = {f'Fold 1': [['C', 'D', 'E'], ['A', 'B']], 
               'Fold 2': [['B', 'D', 'E'], ['A', 'C']],
               'Fold 3': [['B', 'C', 'E'], ['A', 'D']],
               'Fold 4': [['B', 'C', 'D'], ['A', 'E']],
               'Fold 5': [['A', 'D', 'E'], ['B', 'C']],
               'Fold 6': [['A', 'C', 'E'], ['B', 'D']],
               'Fold 7': [['A', 'C', 'D'], ['B', 'E']],
               'Fold 8': [['A', 'B', 'E'], ['C', 'D']],
               'Fold 9': [['A', 'B', 'D'], ['C', 'E']],
               'Fold 10': [['A', 'B', 'C'], ['D', 'E']]}

In [ ]:
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    
    data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
    data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
    data['area'] = data['attribute_2'] * data['attribute_3']

    feature = [f for f in df_test.columns if f.startswith('measurement') or f=='loading']

    
    full_fill_dict = dict()
    full_fill_dict['measurement_17'] = {
        'A': ['measurement_5','measurement_6','measurement_8'],
        'B': ['measurement_4','measurement_5','measurement_7'],
        'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
        'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
        'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_7','measurement_8']
    }

    
    col = [col for col in df_test.columns if 'measurement' not in col]+ ['loading','m3_missing','m5_missing']
    a = []
    b =[]
    for x in range(3,17):
        corr = np.absolute(data.drop(col, axis=1).corr()[f'measurement_{x}']).sort_values(ascending=False)
        a.append(np.round(np.sum(corr[1:4]),3))
        b.append(f'measurement_{x}')
    c = pd.DataFrame()
    c['Selected columns'] = b
    c['correlation total'] = a
    c = c.sort_values(by = 'correlation total',ascending=False).reset_index(drop = True)

    for i in range(10):
        measurement_col = 'measurement_' + c.iloc[i,0][12:]
        fill_dict = {}
        for x in data.product_code.unique() : 
            corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
            measurement_col_dic = {}
            measurement_col_dic[measurement_col] = corr[1:5].index.tolist()
            fill_dict[x] = measurement_col_dic[measurement_col]
        full_fill_dict[measurement_col] =fill_dict

    feature = [f for f in data.columns if f.startswith('measurement') or f=='loading']
    nullValue_cols = [col for col in df_train.columns if df_train[col].isnull().sum()!=0]

    for code in data.product_code.unique():
        total_na_filled_by_linear_model = 0
        for measurement_col in list(full_fill_dict.keys()):
            tmp = data[data.product_code == code]
            column = full_fill_dict[measurement_col][code]
            tmp_train = tmp[column+[measurement_col]].dropna(how='any')
            tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]

            model = HuberRegressor(epsilon=1.9)
            model.fit(tmp_train[column], tmp_train[measurement_col])
            data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[column])
            print(f'{measurement_col} : {len(tmp_test)}')
            total_na_filled_by_linear_model += len(tmp_test)

        # non-numeric columns:
        NA = data.loc[data["product_code"] == code,nullValue_cols ].isnull().sum().sum()
        model1 = KNNImputer(n_neighbors=5)
        data.loc[data.product_code==code, feature] = model1.fit_transform(data.loc[data.product_code==code, feature])

    data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
    data['measurement_std'] = data[[f'measurement_{i}' for i in range(3, 17)]].std(axis=1)
    data['measurement_median'] = data[[f'measurement_{i}' for i in range(3, 17)]].median(axis=1)
    data['measurement_max'] = data[[f'measurement_{i}' for i in range(3, 17)]].max(axis=1)
    data['measurement_min'] = data[[f'measurement_{i}' for i in range(3, 17)]].min(axis=1)
    data['measurement_skew'] = data[[f'measurement_{i}' for i in range(3, 17)]].skew(axis=1)
    
    df_train = data.iloc[:df_train.shape[0],:]
    df_test = data.iloc[df_train.shape[0]:,:]

    woe_encoder = WoEEncoder(variables=['attribute_0'])
    woe_encoder.fit(df_train, df_train['failure'])
    df_train = woe_encoder.transform(df_train)
    df_test = woe_encoder.transform(df_test)

    df_train = df_train.drop(columns=['measurement_std', 'measurement_median', 'measurement_max', 'measurement_min', 'measurement_skew'])
    df_test = df_test.drop(columns=['measurement_std', 'measurement_median', 'measurement_max', 'measurement_min', 'measurement_skew'])

    return df_train, df_test

In [ ]:
df_train, df_test = preprocessing(train, test)

Columns selected by correlation sum of the 3 first rows : 


,Selected columns,correlation total
0,measurement_8,0.454
1,measurement_11,0.395
2,measurement_5,0.386
3,measurement_6,0.365
4,measurement_7,0.336
5,measurement_4,0.331
6,measurement_15,0.301
7,measurement_10,0.300
8,measurement_16,0.252
9,measurement_14,0.225



-------- Product code A ----------

filled by linear model :
measurement_17 : 386
measurement_8 : 167
measurement_11 : 225
measurement_5 : 113


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


measurement_6 : 146
measurement_7 : 153


...

measurement_5 : 119


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


measurement_6 : 132
measurement_7 : 136


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


measurement_4 : 89
measurement_15 : 350
measurement_10 : 246
measurement_16 : 294
measurement_14 : 283

2427 filled by linear model 
1402 filled by KNN 


In [ ]:
features = ['loading', 'attribute_0', 'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'area', 'm3_missing', 'm5_missing', 'measurement_avg']

In [ ]:
NUM_FEATURE = len(features)
DROPOUT_RATE = 0.2

In [ ]:
# [ref] https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/discussion/349385
def get_stacked_dense(x, cur_dim, stack_depth, activation):
    """
    Parameters
    ----------
        `x`: current layer
        `cur_dim`: the number of dimensions of the first stacked layer
        `stack_depth`: the number of stacked layers
        `activation`: the activation function used for the dense layers

    Returns
    -------
        `x`: the output of the stacked layers
    """
    for i in range(stack_depth):
        x = Dense(cur_dim-i, activation)(x)
    return x

def get_triple_stacked(x_0, start_dim):
    """
    Parameters
    ----------
        `x_0`: current layer
        `start_dim`: the number of dimensions of the first stacked layer

    Returns
    -------
        `x_00000, x_00001, x_00010, x_00011, x_00100, x_00101, x_00110, x_00111`: the outputs of the stacked layers
    """
    x_00 = Dense(start_dim, swish)(x_0)
    x_000 = Dense(start_dim-1, swish)(x_00)
    x_0000 = Dense(start_dim-2, swish)(x_000)

    x_00000 = get_stacked_dense(x_0000, start_dim-3, 2, swish)
    x_00000 = BatchNormalization()(x_00000)

    x_00001 = get_stacked_dense(x_0000, start_dim-4, 2, swish)
    x_00001 = BatchNormalization()(x_00001)

    x_0001 = Dense(start_dim-3, swish)(x_000)
    x_00010 = get_stacked_dense(x_0001, start_dim-4, 2, swish)
    x_00010 = BatchNormalization()(x_00010)

    x_00011 = get_stacked_dense(x_0001, start_dim-5, 2, swish)
    x_00011 = BatchNormalization()(x_00011)

    x_001 = Dense(start_dim-2, swish)(x_00)
    x_0010 = Dense(start_dim-3, swish)(x_001)
    x_00100 = get_stacked_dense(x_0010, start_dim-4, 2, swish)
    x_00100 = BatchNormalization()(x_00100)

    x_00101 = get_stacked_dense(x_0010, start_dim-5, 2, swish)
    x_00101 = BatchNormalization()(x_00101)

    x_0011 = Dense(start_dim-4, swish)(x_001)
    x_00110 = get_stacked_dense(x_0011, start_dim-5, 2, swish)
    x_00110 = BatchNormalization()(x_00110)

    x_00111 = get_stacked_dense(x_0011, start_dim-6, 2, swish)
    x_00111 = BatchNormalization()(x_00111)

    return x_00000, x_00001, x_00010, x_00011, x_00100, x_00101, x_00110, x_00111

def get_model(feature_num):
    """
    Parameters
    ----------
        `feature_num`: the number of the features from the training dataset

    Returns
    -------
        `model`: the assembled model
    """
    inputs = Input(shape=(feature_num,))
    x = Dense(20, swish)(inputs)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT_RATE)(x)
    x_0 = Dense(20, swish)(x)
    x_00000, x_00001, x_00010, x_00011, x_00100, x_00101, x_00110, x_00111 = get_triple_stacked(x_0, 19)
    x_01000, x_01001, x_01010, x_01011, x_01100, x_01101, x_10110, x_01111 = get_triple_stacked(x_0, 18)
    cat_layer_list = [x_00000, x_00001, x_00010, x_00011, x_00100, x_00101, x_00110, x_00111, x_01000, x_01001, x_01010, x_01011, x_01100, x_01101, x_10110, x_01111]
    cat = concatenate(cat_layer_list)
    dense1 = Dense(20, swish)(cat)
    output = Dense(1, sigmoid)(dense1)
    model = Model(inputs, output)
    return model

In [ ]:
ES = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=12, mode="min", restore_best_weights=True, verbose=2)
LR = tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=5, mode="min", restore_best_weights=True, min_lr=1e-12, verbose=2)

In [ ]:
test_predictions = np.zeros((df_test.shape[0], 1))
total_folds = len(folds_dict.keys())
for i, fold in enumerate(folds_dict.keys()):
    
    print(fold)
    
    x_train, y_train = df_train[df_train['product_code'].isin(folds_dict[fold][0])][features].values, df_train[df_train['product_code'].isin(folds_dict[fold][0])]['failure'].values
    x_val, y_val = df_train[df_train['product_code'].isin(folds_dict[fold][1])][features].values, df_train[df_train['product_code'].isin(folds_dict[fold][1])]['failure'].values

    model = get_model(NUM_FEATURE)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss='binary_crossentropy', metrics=['binary_accuracy'])
    history = model.fit(x_train, y_train,
                        batch_size=32,
                        epochs=300,
                        validation_data=(x_val, y_val),
                        callbacks=[ES, LR],
                        verbose=1)
    history_df = pd.DataFrame(history.history)
    print(f"Min val loss: {min(history_df['val_loss'].values)}")
    model.save(f"model{i}.h5")
    y_pred = model.predict(df_test[features].values)
    test_predictions += y_pred / total_folds

Fold 1
Epoch 1/300
507/507 [==============================] - 18s 17ms/step - loss: 0.5238 - binary_accuracy: 0.7846 - val_loss: 0.5168 - val_binary_accuracy: 0.7864 - lr: 0.0100
Epoch 2/300
507/507 [==============================] - 8s 16ms/step - loss: 0.5117 - binary_accuracy: 0.7879 - val_loss: 0.5158 - val_binary_accuracy: 0.7864 - lr: 0.0100
Epoch 3/300
507/507 [==============================] - 8s 15ms/step - loss: 0.5112 - binary_accuracy: 0.7880 - val_loss: 0.5107 - val_binary_accuracy: 0.7864 - lr: 0.0100
Epoch 4/300
507/507 [==============================] - 7s 14ms/step - loss: 0.5118 - binary_accuracy: 0.7880 - val_loss: 0.5123 - val_binary_accuracy: 0.7864 - lr: 0.0100
Epoch 5/300
507/507 [==============================] - 7s 14ms/step - loss: 0.5120 - binary_accuracy: 0.7880 - val_loss: 0.5114 - val_binary_accuracy: 0.7864 - lr: 0.0100
Epoch 6/300
507/507 [==============================] - 8s 15ms/step - loss: 0.5113 - binary_accuracy: 0.7880 - val_loss: 0.5209 - val_bin

In [ ]:
sub = pd.DataFrame({'id':df_test['id'], 'failure':test_predictions.reshape(-1)})
sub.to_csv('TF_NN_cross.csv', index=False)

In [ ]:
!kaggle competitions submit -c tabular-playground-series-aug-2022 -f TF_NN_cross.csv -m "Tensorflow NN cross"

100% 520k/520k [00:03<00:00, 160kB/s]
Successfully submitted to Tabular Playground Series - Aug 2022

In [ ]:
!zip models.zip ./*.h5

  adding: model0.h5 (deflated 71%)
  adding: model1.h5 (deflated 71%)
  adding: model2.h5 (deflated 71%)
  adding: model3.h5 (deflated 71%)
  adding: model4.h5 (deflated 71%)
  adding: model5.h5 (deflated 71%)
  adding: model6.h5 (deflated 71%)
  adding: model7.h5 (deflated 71%)
  adding: model8.h5 (deflated 71%)
  adding: model9.h5 (deflated 71%)
